<a href="https://colab.research.google.com/github/mdahmadullahmahmood/Text-Summarization-Model/blob/main/Text_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers streamlit bert-extractive-summarizer sentencepiece torch joblib pymupdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20

In [6]:
from transformers import pipeline
from summarizer import Summarizer


# Load models from Hugging Face
bart_summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
t5_summarizer = pipeline("summarization", model="t5-large")
bert_summarizer = Summarizer()

print("✅ Models loaded successfully!")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

✅ Models loaded successfully!


In [7]:
%%writefile app.py
import streamlit as st
from transformers import pipeline
from summarizer import Summarizer  # Corrected import
import fitz  # PyMuPDF for PDF support
import os

# Initialize models for summarization
@st.cache_resource
def load_bart_model():
    return pipeline("summarization", model="facebook/bart-large-cnn", device=0)

@st.cache_resource
def load_t5_model():
    return pipeline("summarization", model="t5-large", device=0)

@st.cache_resource
def load_bert_extractive_model():
    return Summarizer()

# Function to extract text from PDF
def extract_text_from_pdf(uploaded_file):
    doc = fitz.open(stream=uploaded_file.read(), filetype="pdf")
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text

# Streamlit UI setup
st.set_page_config(page_title="Text Summarization Tool 📝", page_icon="📝", layout="wide")

# Custom CSS for better UI
st.markdown("""
    <style>
        .title { text-align: center; font-size: 36px; color: #1a1a1a; }
        .description { text-align: center; font-size: 20px; color: #555; margin-bottom: 50px; }
        .summary-box { border: 2px solid #4CAF50; padding: 20px; background-color: #f1f1f1; border-radius: 10px; margin-top: 30px; }
    </style>
""", unsafe_allow_html=True)

# App header
st.markdown('<div class="title">Text Summarization Tool 📝</div>', unsafe_allow_html=True)
st.markdown('<div class="description">Summarize any text or PDF effortlessly using NLP-based models.</div>', unsafe_allow_html=True)

# Sidebar options
st.sidebar.title("Settings")
min_length = st.sidebar.slider("Minimum Summary Length", 50, 200, 100)
max_length = st.sidebar.slider("Maximum Summary Length", 150, 500, 300)

# Select summarization model
model_choice = st.sidebar.selectbox("Choose Summarization Model", ["BART", "T5", "Extractive (BERT)"])

# Load the selected model
if model_choice == "BART":
    summarizer = load_bart_model()
elif model_choice == "T5":
    summarizer = load_t5_model()
elif model_choice == "Extractive (BERT)":
    summarizer = load_bert_extractive_model()

# File upload for text input
uploaded_file = st.file_uploader("Upload a text or PDF file", type=["txt", "pdf"])

if uploaded_file is not None:
    if uploaded_file.name.endswith(".pdf"):
        file_text = extract_text_from_pdf(uploaded_file)
    else:
        file_text = uploaded_file.read().decode("utf-8")
    st.text_area("Uploaded Text", file_text, height=300)
else:
    input_text = st.text_area("Enter your text here for summarization", height=300)

# Button to generate summary
if st.button("Summarize"):
    if uploaded_file is not None or input_text.strip():
        with st.spinner("Summarizing..."):
            text_to_summarize = file_text if uploaded_file else input_text

            if model_choice == "Extractive (BERT)":
                summary_text = summarizer(text_to_summarize, min_length=min_length)
            else:
                summary = summarizer(text_to_summarize, max_length=max_length, min_length=min_length, do_sample=False)
                summary_text = summary[0]['summary_text']

            # Display summary
            st.markdown('<div class="summary-box">', unsafe_allow_html=True)
            st.markdown('<div class="summary-header">Summary:</div>', unsafe_allow_html=True)
            st.markdown(f'<div class="summary-text">{summary_text}</div>', unsafe_allow_html=True)
            st.markdown('</div>', unsafe_allow_html=True)

            # Option to download summary
            summary_filename = "summary.txt"
            with open(summary_filename, "w") as f:
                f.write(summary_text)
            st.download_button("Download Summary", data=open(summary_filename, "r").read(), file_name=summary_filename)
    else:
        st.warning("Please input some text or upload a file to summarize.")

# Sidebar information
st.sidebar.markdown("### About this Tool")
st.sidebar.write("This tool uses pre-trained models (BART, T5, and BERT) from Hugging Face to summarize text or PDF content efficiently.")

# Footer
st.markdown("""
    <div style="text-align: center; margin-top: 50px; color: #888; font-size: 14px;">
        Built with ❤️ MD AHMADULLAH MAHMOOD | Powered by Hugging Face & Streamlit
    </div>
""", unsafe_allow_html=True)


Writing app.py


In [8]:
!wget -q -O - ipv4.icanhazip.com # this will generate the local tunnel code

34.106.56.232


In [13]:
!streamlit run app.py & npx localtunnel --port 8501 # this will generate the streamlit app



⠙⠹⠸
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8502
  Network URL: http://172.28.0.12:8502
  External URL: http://34.106.56.232:8502

⠼⠴Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋your url is: https://huge-carrots-travel.loca.lt
  Stopping...
^C
